In [1]:
import shapely#.geometry import Point, LineString
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np
import random

In [5]:
gdf = gpd.read_file("./MEX/MEX_ADM2_fixedInternalTopology.shp")
gdf = gdf[['shapeID', 'geometry']]

match = pd.read_csv("./gB_IPUMS_match.csv")
match = match[['shapeID', 'MUNI2015']]
ref_dict = dict(zip(match['MUNI2015'], match['shapeID']))

df = pd.read_csv("./mexico2010.csv")
df = df[['GEO2_MX', 'sum_income', 'total_pop', 'unrel_ppl', 'perc_urban', 'sum_num_intmig']]
df['GEO2_MX'] = df['GEO2_MX'].astype(str).str.replace("484", "").astype(int).map(ref_dict)
df = df.rename(columns = {'GEO2_MX': 'shapeID'})

g
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: geometries must not contain null elements

IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: geometries must not contain null elements

IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Sh

In [4]:
class GeoPath():

    def __init__(self, sending_id, recieving_id, gdf, df, x = False, degrees = 1):

        self.gdf = gdf


    def calc_centroid_x(self, x):
        return x.centroid.x
    
    def calc_centroid_y(self, x):
        return x.centroid.y

    def calc_shortest_line(self, xs, ys):

        x1, x2 = xs
        y1, y2 = ys

        line = shapely.geometry.LineString([shapely.geometry.Point(x1, y1), shapely.geometry.Point(x2, y2)]).wkt

        # Convert new_df to a geodataframe
        new_row = pd.DataFrame(columns = ['shapeID', 'geometry', 'c_long', 'c_lat'])
        new_row['shapeID'], new_row['geometry'], new_row['c_long'], new_row['c_lat'] = ['line'], [line], [0], [0]
        
        new_row['geometry'] = new_row['geometry'].apply(shapely.wkt.loads)
        new_row = gpd.GeoDataFrame(new_row, geometry='geometry')

        
        
        return new_row


    def show(self):


        gdf_temp = self.gdf[self.gdf['shapeID'].isin([sending_id, recieving_id])]
        gdf_temp['c_long'] = gdf_temp['geometry'].apply(lambda x: self.calc_centroid_x(x))
        gdf_temp['c_lat'] = gdf_temp['geometry'].apply(lambda x: self.calc_centroid_y(x))
        line = self.calc_shortest_line(gdf_temp['c_long'].to_list(), gdf_temp['c_lat'].to_list()).buffer(.001)

        print(line)

        # print(self.gdf.geometry[self.gdf.geometry(line.intersects(self.gdf.geometry))])

        print(self.gdf.geometry)
        # print(line.geometry)

        inp, res = self.gdf.sindex.query_bulk(line.geometry, predicate='intersects')
        points['intersects'] = np.isin(np.arange(0, len(points)), inp)

        # yo = gpd.GeoSeries(self.gdf.geometry).intersects(other = line, align = False)
        print(inp, res)
        # yo = pd.DataFrame(yo)
        # yo.columns = ['shape']
        # yo[yo['shape'] == True].reset_index()['index'].values
        # print(yo[yo['shape'] == True].reset_index()['index'].values)
        # line

        # print(line.buffer(1))

        plt.figure(1,figsize=(10, 10)) 
        base = gdf_temp.plot(edgecolor='black')
        line.plot(ax = base, marker='o', color='red', markersize = 5)

        return yo

    



sending_id = random.choice(df['shapeID'].to_list())
recieving_id = random.choice(df['shapeID'].to_list())

degrees = random.randint(1, 4)

g = GeoPath(sending_id, recieving_id, gdf, df, degrees = degrees).show()


ValueError: Null geometry supports no operations

In [100]:
gd = pd.DataFrame(g)
gd.columns = ['shape']


gd[gd['shape'] == True].reset_index()['index'].values
# gd
# gd['shape'].unique()

array([], dtype=int64)

In [60]:
gdf.shape

(2458, 2)

In [96]:
from shapely.geometry import Polygon, LineString, Point
s = gpd.GeoSeries(
    [
        Polygon([(0, 0), (2, 2), (0, 2)]),
        LineString([(0, 0), (2, 2)]),
        LineString([(2, 0), (0, 2)]),
        Point(0, 1),
    ],
)
s2 = gpd.GeoSeries(
    [
        LineString([(1, 0), (1, 3)]),
        LineString([(2, 0), (0, 2)]),
        Point(1, 1),
        Point(0, 1),
    ],
    index=range(1, 5),
)

In [97]:
line = LineString([(-1, 1), (3, 1)])

In [98]:
s.intersects(line)

0    True
1    True
2    True
3    True
dtype: bool